In [1]:
# Packages
import dask.dataframe as dd
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

In [41]:
import psutil

# Obtenir les informations sur la mémoire
memory_info = psutil.virtual_memory()

# Afficher la mémoire totale, utilisée et disponible en GB
total_memory = memory_info.total / (1024 ** 3)
used_memory = memory_info.used / (1024 ** 3)
available_memory = memory_info.available / (1024 ** 3)

print(f'Mémoire Totale: {total_memory:.2f} GB')
print(f'Mémoire Utilisée: {used_memory:.2f} GB')
print(f'Mémoire Disponible: {available_memory:.2f} GB')

Mémoire Totale: 15.40 GB
Mémoire Utilisée: 5.74 GB
Mémoire Disponible: 9.66 GB


In [3]:
MovieLens_Imdb2_Reduced_path = 'C:/Users/anacv/OneDrive/Documentos/Data Scientist/00. Projet/Data/DataFrames travaillés/1. Unions/MovieLens_Imdb/MovieLens_Imdb2_DirectorsInfoReduced2.feather'
Actors_path = 'C:/Users/anacv/OneDrive/Documentos/Data Scientist/00. Projet/Data/DataFrames travaillés/1. Unions/MovieLens_Imdb/0_Actors(option2).feather'

In [ ]:
### Part III : Union avec la base de données principales

# Convertir chaque fichier Feather en Parquet
#pd.read_feather(MovieLens_Imdb2_DirectorsInfo_path).to_parquet('MovieLens_Imdb2_DirectorsInfo.parquet')

In [5]:
Union = dd.merge(
    pd.read_feather(MovieLens_Imdb2_Reduced_path), 
    pd.read_feather(Actors_path),
    how='left',
    left_on='imdbId',
    right_on='imdbId'
)

In [11]:
Union.info()

<class 'dask_expr.DataFrame'>
Columns: 18 entries, userId to actors_knownForTitles_primaryTitle
dtypes: Int64(3), datetime64[ns](1), float64(3), int64(3), string(8)

In [13]:
Union.shape[0].compute()

20000263

In [15]:
Union.columns

Index(['userId', 'movieId', 'rating', 'date_rating', 'genres', 'title_name',
       'release_year', 'imdbId', 'imdb_tconst', 'imdb_isAdult',
       'imdb_runtimeMinutes', 'imdb_averageRating', 'imdb_numVotes',
       'directors_primaryName', 'directors_knownForTitles_primaryTitle',
       'actors_characters', 'actors_primaryName',
       'actors_knownForTitles_primaryTitle'],
      dtype='object')

In [17]:
# Suppression des colonnes 'date_rating' et 'imdb_runtimeMinutes' de 'union'
Union = Union.drop(columns=['date_rating', 'imdb_runtimeMinutes', 'imdb_tconst', 'imdb_isAdult'])

In [19]:
# Calcul de la taille en mémoire du DataFrame 'union' en octets
memory_usage = Union.memory_usage(deep=True).sum().compute()

# Conversion en mégaoctets (optionnel)
memory_usage_mb = memory_usage / (1024 ** 2)
print(f"Le DataFrame 'union' pèse environ : {memory_usage_mb:.2f} Mo")

Le DataFrame 'union' pèse environ : 9402.99 Mo


In [23]:
# Exporter le dask DataFrame au format Parquet, en utilisant des partitions
Union.to_parquet(r'C:\Users\anacv\OneDrive\Documentos\Data Scientist\00. Projet\Data\DataFrames travaillés\1. Unions\MovieLens_Imdb\Union_option2',
                 engine='pyarrow', write_index=False)

In [25]:
Union.head(5)

,userId,movieId,rating,genres,title_name,release_year,imdbId,imdb_averageRating,imdb_numVotes,directors_primaryName,directors_knownForTitles_primaryTitle,actors_characters,actors_primaryName,actors_knownForTitles_primaryTitle
0,1,2,3.5,Adventure|Children|Fantasy,Jumanji,1995,113497,7.1,384991,Joe Johnston,"Captain America: The First Avenger,The Rockete...","[""Alan Parrish""],[""Judy Shepherd""],[""Sarah Whi...","Robin Williams,Kirsten Dunst,Bonnie Hunt","Mrs. Doubtfire,Good Morning, Vietnam,One Hour ..."
1,1,29,3.5,Adventure|Drama|Fantasy|Mystery|Sci-Fi,"City of Lost Children, The (Cité des enfants p...",1995,112682,7.5,72399,"Marc Caro,Jean-Pierre Jeunet","Delicatessen,The City of Lost Children,Alien R...","[""One""],[""Krank""],[""Miette""]","Ron Perlman,Daniel Emilfork,Judith Vittet","Hand of God,Hellboy II: The Golden Army,Hellbo..."
2,1,32,3.5,Mystery|Sci-Fi|Thriller,Twelve Monkeys (a.k.a. 12 Monkeys),1995,114746,8.0,654005,Terry Gilliam,"Brazil,Monty Python and the Holy Grail,The Ima...","[""James Cole""],[""Kathryn Railly""],[""Jeffrey Go...","Bruce Willis,Madeleine Stowe,Brad Pitt","Die Hard,Pulp Fiction,Moonlighting,The Sixth S..."
3,1,47,3.5,Mystery|Thriller,Seven (a.k.a. Se7en),1995,114369,8.6,1839918,David Fincher,"Se7en,Zodiac,Gone Girl,The Social Network","[""Somerset""],[""Mills""],[""John Doe""]","Morgan Freeman,Brad Pitt,Kevin Spacey","Se7en,Driving Miss Daisy,Million Dollar Baby,T..."
4,1,50,3.5,Crime|Mystery|Thriller,"Usual Suspects, The",1995,114814,8.5,1161903,Bryan Singer,"X-Men,The Usual Suspects,Superman Returns,X-Me...","[""Verbal""],[""Keaton""],[""Dave Kujan""]","Kevin Spacey,Gabriel Byrne,Chazz Palminteri","L.A. Confidential,The Usual Suspects,American ..."


In [ ]:
### Pour la lecture :

union_loaded = dd.read_parquet(r'C:\Users\anacv\OneDrive\Documentos\Data Scientist\00. Projet\Data\DataFrames travaillés\1. Unions\MovieLens_Imdb\Union_option2')



In [23]:
        ### Pour libèrer du mémoire : Sauvegarde du DataFrame 'Union'

# Vérifier l'utilisation de la mémoire en bytes
memory_usage_bytes = Union.memory_usage(deep=True).sum()




In [27]:
# Convertir en mégaoctets (MB)
memory_usage_MB = memory_usage_bytes / 1024**2

print(f'Le DataFrame Union pèse environ {memory_usage_MB:.2f} MB en mémoire.')

# Sauvegarder le fichier
#Union.to_feather(r'C:\Users\anacv\OneDrive\Documentos\Data Scientist\00. Projet\Data\DataFrames travaillés\1. Unions\MovieLens_Imdb\Union.feather')

TypeError: unsupported format string passed to Scalar.__format__

In [43]:
# Sauvegarder le fichier
# Augmenter le nombre de partitions pour réduire la taille de chaque écriture
Union = Union.repartition(npartitions=1000)
Union.to_parquet(
    r'C:\Users\anacv\OneDrive\Documentos\Data Scientist\00. Projet\Data\DataFrames travaillés\1. Unions\MovieLens_Imdb\Union_parquet',
    compression='gzip'  # ou snappy
)


KeyboardInterrupt: 